In [1]:
import pandas as pd
import os.path as op
import retro
from src.features.annotations import create_runevents, trim_events_df
from src.features.features import compute_framewise_aps
import matplotlib.pyplot as plt
from src.params import figures_path
from nilearn import plotting
from nilearn import image
import os
import numpy as np
from nilearn.plotting import plot_design_matrix
import load_confounds
from nilearn.input_data import NiftiMasker
from nilearn.glm.first_level import FirstLevelModel
import pickle
import nilearn
from src.data.data import retrieve_scanvariables
from src.params import actions

# set constants
ses = 'ses-001'
sub = 'sub-01'
dpath = '/media/hyruuk/Seagate Expansion Drive/DATA/shinobi/' # corresponds to the data/ folder in which the files are in bids format
t_r = 1.49

In [2]:
runs = ['2', '3'] # These two runs have the same number of confounds, that whole cell runs without issue

# load the data and create design matrices
fmri_imgs = []
design_matrices = []
for run in runs:
    print('Loading run {}'.format(run))
    # generate filenames
    events_fname = dpath + '{}/{}/func/{}_{}_task-shinobi_run-0{}_events.tsv'.format(sub, ses, sub, ses, run)
    filename = dpath + 'derivatives/fmriprep-20.2lts/fmriprep/{}/{}/func/{}_{}_task-shinobi_run-{}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'.format(sub, ses, sub, ses, run)
    confounds_fname = dpath + 'derivatives/fmriprep-20.2lts/fmriprep/{}/{}/func/{}_{}_task-shinobi_run-{}_desc-confounds_timeseries.tsv'.format(sub, ses, sub, ses, run)
    
    # Obtain list of bk2 files from events
    startevents = pd.read_table(events_fname)
    files = startevents['stim_file'].values.tolist()
    files = [dpath + file for file in files]

    # Retrieve variables from these files and create events_df based on annotations
    runvars = retrieve_scanvariables(files)
    events_df = create_runevents(runvars, startevents, actions=actions)
    events_df['trial_type'].unique()
    LvR_df = trim_events_df(events_df, trim_by = 'LvR')

    fmri_imgs.append(image.concat_imgs(filename))
    
    # Load the confounds names and confounds dataframe
    conf=load_confounds.Params36()
    conf.load(confounds_fname)
    print('Confounds shape of run {} : {}'.format(run, conf.confounds_.shape))
    confounds = pd.DataFrame.from_records(load_confounds.Params36().load(confounds_fname)) # no idea why but I can't use conf directly in the DM
    
    
    # create design matrices
    n_slices = confounds.shape[0]
    frame_times = np.arange(n_slices) * t_r
    design_matrix = nilearn.glm.first_level.make_first_level_design_matrix(frame_times,
                                                                           events=LvR_df,
                                                                          drift_model=None,
                                                                          add_regs=confounds,
                                                                          add_reg_names=conf.columns_)
    design_matrices.append(design_matrix)
    

# Create and fit model    
fmri_glm = FirstLevelModel(t_r=1.49,
                           noise_model='ar1',
                           standardize=False,
                           hrf_model='spm',
                           drift_model=None,
                           high_pass=.01,
                           n_jobs=-1,
                           smoothing_fwhm=5)
fmri_glm = fmri_glm.fit(fmri_imgs, design_matrices=design_matrices)

# Compute contrasts
fmri_glm.generate_report(contrasts=['LeftH-RightH'])

Loading run 2


/home/hyruuk/anaconda3/envs/hyruuk_shinobi_behav/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/home/hyruuk/anaconda3/envs/hyruuk_shinobi_behav/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Confounds shape of run 2 : (486, 46)
Loading run 3
Confounds shape of run 3 : (485, 46)


/home/hyruuk/anaconda3/envs/hyruuk_shinobi_behav/lib/python3.8/site-packages/nilearn/glm/first_level/first_level.py:580: UserWarning: One contrast given, assuming it for all 2 runs
  warn('One contrast given, assuming it for all %d runs' % n_runs)


In [3]:
#### DOESN'T WORK, number of confounds don't match
runs = ['1', '2'] 

# load the data and create design matrices
fmri_imgs = []
design_matrices = []
for run in runs:
    print('Loading run {}'.format(run))
    # generate filenames
    events_fname = dpath + '{}/{}/func/{}_{}_task-shinobi_run-0{}_events.tsv'.format(sub, ses, sub, ses, run)
    filename = dpath + 'derivatives/fmriprep-20.2lts/fmriprep/{}/{}/func/{}_{}_task-shinobi_run-{}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'.format(sub, ses, sub, ses, run)
    confounds_fname = dpath + 'derivatives/fmriprep-20.2lts/fmriprep/{}/{}/func/{}_{}_task-shinobi_run-{}_desc-confounds_timeseries.tsv'.format(sub, ses, sub, ses, run)
    
    # Obtain list of bk2 files from events
    startevents = pd.read_table(events_fname)
    files = startevents['stim_file'].values.tolist()
    files = [dpath + file for file in files]

    # Retrieve variables from these files and create events_df based on annotations
    runvars = retrieve_scanvariables(files)
    events_df = create_runevents(runvars, startevents, actions=actions)
    events_df['trial_type'].unique()
    LvR_df = trim_events_df(events_df, trim_by = 'LvR')

    fmri_imgs.append(image.concat_imgs(filename))
    
    # Load the confounds names and confounds dataframe
    conf=load_confounds.Params36()
    conf.load(confounds_fname)
    print('Confounds shape of run {} : {}'.format(run, conf.confounds_.shape))
    confounds = pd.DataFrame.from_records(load_confounds.Params36().load(confounds_fname)) # no idea why but I can't use conf directly in the DM
    
    
    # create design matrices
    n_slices = confounds.shape[0]
    frame_times = np.arange(n_slices) * t_r
    design_matrix = nilearn.glm.first_level.make_first_level_design_matrix(frame_times,
                                                                           events=LvR_df,
                                                                          drift_model=None,
                                                                          add_regs=confounds,
                                                                          add_reg_names=conf.columns_)
    design_matrices.append(design_matrix)
    

# Create and fit model    
fmri_glm = FirstLevelModel(t_r=1.49,
                           noise_model='ar1',
                           standardize=False,
                           hrf_model='spm',
                           drift_model=None,
                           high_pass=.01,
                           n_jobs=-1,
                           smoothing_fwhm=5)
fmri_glm = fmri_glm.fit(fmri_imgs, design_matrices=design_matrices)

# Compute contrasts
fmri_glm.generate_report(contrasts=['LeftH-RightH'])

Loading run 1
Confounds shape of run 1 : (425, 44)
Loading run 2


/home/hyruuk/anaconda3/envs/hyruuk_shinobi_behav/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/home/hyruuk/anaconda3/envs/hyruuk_shinobi_behav/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Confounds shape of run 2 : (486, 46)


/home/hyruuk/anaconda3/envs/hyruuk_shinobi_behav/lib/python3.8/site-packages/nilearn/glm/first_level/first_level.py:580: UserWarning: One contrast given, assuming it for all 2 runs
  warn('One contrast given, assuming it for all %d runs' % n_runs)


ValueError: t contrasts should be length P=49, but this is length 47